In [1]:
#%pip install beautifulsoup4
#%pip install pip-system-certs

In [6]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
from langchain_community.graphs import Neo4jGraph
from PyPDF2 import PdfReader
from langdetect import detect
import spacy
import re
from unidecode import unidecode
import time

Conectando ao Neo4j 

In [7]:
from neo4j import GraphDatabase

# Neo4j variables
NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME =os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")


graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

- Buscar os links para a tese no Neo4j
- Buscar o link do PDF no repositório da universidade
- Baixar o PDF
- Extrair o texto do PDF 

Buscar links para a tese no Neo4j

In [11]:
# Query to get theses and their repositories from the Neo4j database
# This query retrieves theses that are published by the Federal University of Bahia (UFBA) and have not been processed yet.
query1 = """
MATCH (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:UFBA'})
WHERE NOT EXISTS {(t{processed:true})}
RETURN distinct(t["uri"]) as Thesis, t["repository"] as repository 
"""

# Alternative query to get theses that do not have excerpts and have been processed.
query2 = """ 
match (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:UFBA'})
where (not (t:Thesis)-[:BFO_0000050]-(:Exerpt)) and  exists {(t{processed:true})}
return distinct(t["uri"]) as Thesis, t["repository"] as repository
"""

# Query to retrive theses with few pages and their repositories.
query3 = """
MATCH (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:UFBA'})
Match (t:Thesis)-[:BFO_0000050]-(e:Exerpt)
RETURN count(e) as pages, t["uri"] as Thesis, t["repository"] as repository
 order by (pages) limit 40
"""

graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
results = graph.execute_query(query1).records
graph.close()

#results = graph.query(query2)
# Convert the results to a DataFrame
#thesis_repository = pd.DataFrame(results, columns=["pages", "Thesis", "repository"])
thesis_repository = pd.DataFrame(results, columns=["Thesis", "repository"])
thesis_repository = thesis_repository.reset_index(drop=True) 
thesis_repository

,Thesis,repository
0,tag:stardog:api:modernizar_as_cidades_civiliza...,http://repositorio.ufba.br/ri/handle/ri/31924
1,tag:stardog:api:a_primeira_doutora_negra_em_qu...,https://repositorio.ufba.br/handle/ri/38945
2,tag:stardog:api:mais_que_um_espaco_religioso_a...,https://repositorio.ufba.br/handle/ri/35056
3,tag:stardog:api:hubert_fichte_e_seu_xango_conf...,http://www.repositorio.ufba.br/ri/handle/ri/11633
4,tag:stardog:api:das_possibilidades_heterotopic...,http://repositorio.ufba.br/ri/handle/ri/26587
...,...,...
294,tag:stardog:api:etnoidentificacoes_na_escola_a...,http://www.repositorio.ufba.br/ri/handle/ri/13007
295,tag:stardog:api:coisa_de_branco_a_queixa_e_a_m...,http://repositorio.ufba.br/ri/handle/ri/26935
296,tag:stardog:api:instituicoes_paramaconicas_na_...,http://repositorio.ufba.br/ri/handle/ri/33298
297,tag:stardog:api:tomar_o_proprio_destino_infanc...,https://repositorio.ufba.br/handle/ri/39009


Criando as utils functions

In [9]:
# Funcao para coletar link do pdf

def get_pdf_link(url):
    
    #Fazer requisição e parsear o arquivo html
    try:
        f = requests.get(url, verify=True).text#, verify=False) 
    except:
        print("Sleeping for 5 seconds before retrying...")
        time.sleep(5)
        try:
            f = requests.get(url, verify=True).text#, verify=False) 
        except:
            print("Sleeping for 30 seconds before retrying...")
            time.sleep(30)
            f = requests.get(url, verify=True).text#, verify=False)    
    soup = bs(f, "html.parser")
    
    #Coletando link para as teses
    link_list = []
    for doc in soup.find_all('a', href=True):
        if (doc['href'].endswith('.pdf') & doc['href'].startswith('/bitstream')):
            path = doc['href']
            prefix_uni = 'https://repositorio.ufba.br' #'https://www.teses.usp.br'
            link = prefix_uni + path
            link_list.append(link)
    if link_list != []:
        return list(set(link_list))
    else:
        return None
    
    #return None

#fazer download do arquivo
def download_file(pdf_link, thesis_uri):


    # NOTE the stream=True parameter below
    try:
        r = requests.get(pdf_link, verify=True, stream=True)
    except:
        print("Sleeping for 5 seconds before retrying...")
        time.sleep(5)
        try:
            r = requests.get(pdf_link, verify=True, stream=True)
        except:
            print("Sleeping for 30 seconds before retrying...")
            time.sleep(30)
            r = requests.get(pdf_link, verify=True, stream=True)
    # raise_for_status() is not needed with stream=True
    # r.raise_for_status()
    with open('tese 1.pdf', 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    return 

#Carregando os modelos SpaCy para inglês e português 
nlp_en = spacy.load("en_core_web_lg")
nlp_pt = spacy.load("pt_core_news_lg")

# Funcão que recebe uma string e a limpar para ficar no formato aceitável para uma URI
def process_uri(x):
    return (re.sub('[^a-zA-Z0-9_ ]', '',
            unidecode(x.strip())
                        .replace(" ", "_")
                        .replace("[","")
                        .replace("]","")
                        .replace("?","")
                        .replace("'","")
                        .lower()))


# Função para extrair texto de uma tese em PDF e identificar entidades
def extratc_text(tese_path):
    
    # Processando o arquivo em PDF
    try:
        reader = PdfReader(tese_path)
    except:
        print(f"Erro ao ler o arquivo {tese_path}. ")
        excerpt = {}
        return excerpt
    number_of_pages = len(reader.pages)
    excerpt = {}
    

    # Iterando por cada página
    for page_number in range(number_of_pages):
        
        
        
        # Extraindo o texto da página
        try:
            if page_number == 0:
                try:
                    page_text = (reader.pages[page_number].extract_text() + '\n ' 
                                + reader.pages[page_number+1].extract_text()[:400])
                except:
                    page_text = reader.pages[page_number].extract_text()
            else:
                
                if page_number == number_of_pages -1:
                    try:
                        page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                                    + reader.pages[page_number].extract_text())
                    except:
                        page_text = reader.pages[page_number].extract_text()
                
                else:
                    try:
                        page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                                    + reader.pages[page_number].extract_text() + '\n ' 
                                    + reader.pages[page_number+1].extract_text()[:400])  
                    except:
                        page_text = reader.pages[page_number].extract_text()
        except:
            page_text = ''
        
        # Limpando page text         
        page_text = page_text.replace("'", "").replace('"', '')
        # Verifica se o último caractere é uma barra invertida
        if page_text != '':
            if page_text[-1] == '\\': 
                page_text = page_text[:-1]
        page_text = re.sub(r"[\ud800-\udfff]", " ", page_text)
        page_text = page_text.encode('utf-8').decode('utf-8')
        
        
        # Detectando o idioma do texto
        try:
            lang = detect(page_text)
            
        except:
            lang = 'Não_detectado'
        
        
        # Extraindo as entidades
        page_extracted = False
        if lang == 'pt':
            doc = nlp_pt(page_text)
            page_extracted = True
        if lang == 'en':
            doc = nlp_en(page_text)
            page_extracted = True
        

        if page_extracted:

            persons = []
            gpes = []
            #orgs = []
            for ent in doc.ents:
                if ent.label_ == "PER": #"PERSON":
                    persons.append(process_uri(ent.text))
                if ent.label_ == "LOC": #"GPE":
                    gpes.append(process_uri(ent.text))
                #if ent.label_ == "ORG":
                #    orgs.append(process_uri(ent.text))
                    
            excerpt[page_number] = {'text': page_text, 
                                    'lang': lang, 
                                    'persons': list(set(persons)), 
                                    'gpes': list(set(gpes)),
                                #'orgs': list(set(orgs))
                                }

    return excerpt

Iterando por todas as URI para extrair o texto e gravar no grafo NEO4j

In [10]:
# Gravar hora de início do processamento
start_time = pd.Timestamp.now()
parcial_time = pd.Timestamp.now()

for i in range(0,len(thesis_repository)):

    # Coletando o link do PDF
    url = thesis_repository['repository'][i]
    thesis_uri = thesis_repository['Thesis'][i][16:]
    pdf_link = get_pdf_link(url)
    if pdf_link is not None:
        # Itereando pelos links de PDF encontrados (em alguns casos há outros materiais além da tese)
        for pdf_num in range(len(pdf_link)):
            print(f"{i} - Downloading and extracting: {thesis_uri} from {pdf_link[pdf_num]}")
            download_file(pdf_link[pdf_num], thesis_uri)

            # Extraindo o texto e entidades
            tese_path = 'tese 1.pdf'
            excerpt = extratc_text(tese_path)

            excerpts = []
            persons = []
            gpes = []

            # Salvando as triplas em CSV
            for n in excerpt.keys():

                excerpt_uri = thesis_repository['Thesis'][i] + '_pdf_' + str(pdf_num) + '_p_' + str(n)
                excerpt_text = excerpt[n]['text']
                # Verifica se o último caractere é uma barra invertida
                #if excerpt_text[-1] == '\\': 
                #    excerpt_text = excerpt_text[:-1]
                

                excerpt_page = 'pdf_' + str(pdf_num) + '_p_' + str(n)
                excerpt_lang = excerpt[n]['lang']
                
                excerpts.append([thesis_repository['Thesis'][i], excerpt_uri, excerpt_text, excerpt_page, excerpt_lang])
                    
                #Criando as relações com as entidades
                for person in excerpt[n]['persons']:
                    persons.append([excerpt_uri, 'tag:stardog:api:' + person])
                
                for gpe in excerpt[n]['gpes']:
                    gpes.append([excerpt_uri, 'tag:stardog:api:' + gpe])
            
            # Salvando os dados em CSV no diretório de importação do Neo4j
            pd.DataFrame(excerpts).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\excerpts1.csv', index=False, header=False)
            pd.DataFrame(persons).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\persons1.csv', index=False, header=False)
            pd.DataFrame(gpes).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\gpes1.csv', index=False, header=False)
            
            #Deletando o arquivo PDF após o processamento
            os.remove('tese 1.pdf')

            print(f"Loading triplas: {thesis_uri}")

            # Importanto os dados para o Neo4j
            #Excerpts
            query = """
            LOAD CSV FROM 'file:///excerpts1.csv'
            AS row
            MATCH  (t:Thesis{uri: row[0]})
            MERGE  (e:Exerpt{
                    uri: row[1],
                    text: row[2],
                    page: row[3],
                    lang: row[4]
                    })-[r:BFO_0000050]-(t);
            """
            graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
            results = graph.execute_query(query)
            graph.close()

            #Persons
            query = """
            LOAD CSV FROM 'file:///persons1.csv'
            AS row
            MATCH  (e:Exerpt{uri: row[0]})
            MERGE  (p:Person{uri: row[1]})
            MERGE  (e)-[r:mentions]-(p);
            """
            graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
            results = graph.execute_query(query)
            graph.close()

            #GPEs
            query = """
            LOAD CSV FROM 'file:///gpes1.csv'
            AS row
            MATCH  (e:Exerpt{uri: row[0]})
            MERGE  (p:Place{uri: row[1]})
            MERGE  (e)-[r:mentions]-(p);
            """
            graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
            results = graph.execute_query(query)
            graph.close()

            # Removendo os arquivos CSV após a importação
            os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\excerpts1.csv')
            os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\persons1.csv')
            os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\gpes1.csv')

            print(f"Total time: {pd.Timestamp.now() - start_time}")
            print(f"Partial time: {pd.Timestamp.now() - parcial_time}")
            # Atualizando a hora parcial
            parcial_time = pd.Timestamp.now()
    else:
        print(f"No PDF link found for {thesis_uri}")

    # gravando no grafo que a tese foi processada
    query = """
    MATCH  (t:Thesis{uri: \'""" + thesis_repository['Thesis'][i] +  """\'})
    SET t.processed = true
    """
    graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
    results = graph.execute_query(query)
    graph.close()

0 - Downloading and extracting: comportamento_sedentario_e_coexistencia_de_comportamentos_de_risco_a_saude_em_adolescentes_brasileiros__pesquisa_nacional_de_saude_do_escolar_pense_2015 from https://repositorio.ufba.br/bitstream/ri/33884/1/DISSERTA%c3%87%c3%83O_FINAL_anexo.pdf
Loading triplas: comportamento_sedentario_e_coexistencia_de_comportamentos_de_risco_a_saude_em_adolescentes_brasileiros__pesquisa_nacional_de_saude_do_escolar_pense_2015
Total time: 0 days 00:00:24.764517
Partial time: 0 days 00:00:24.764517
1 - Downloading and extracting: redes_neurais_artificiais_como_ferramenta_de_auxilio_na_operacao_e_controle_da_estacao_de_tratamento_de_agua_de_palmeira_dos_indios from https://repositorio.ufba.br/bitstream/ri/25897/1/Disserta%c3%a7%c3%a3o_Wilson_Mendes_Cavalcante.pdf
Loading triplas: redes_neurais_artificiais_como_ferramenta_de_auxilio_na_operacao_e_controle_da_estacao_de_tratamento_de_agua_de_palmeira_dos_indios
Total time: 0 days 00:00:38.701508
Partial time: 0 days 00:00:1

c:\Users\facordei\OneDrive - Capgemini\Documents\GitHub\Indigenous-Slavery-KG\.venv-IndigSlave\Lib\site-packages\PyPDF2\_cmap.py:142: PdfReadWarning: Advanced encoding /StandardEncoding not implemented yet
  warnings.warn(


Loading triplas: diagnostico_da_necessidade_de_protese_dentaria_em_indios_kiriri
Total time: 0 days 00:38:10.680619
Partial time: 0 days 00:00:16.920089
53 - Downloading and extracting: gasto_energetico_em_repouso_de_um_grupo_de_pacientes_com_hepatite_cronica_pelo_virus_c_sem_tratamento from https://repositorio.ufba.br/bitstream/ri/40199/1/Disserta%c3%a7%c3%a3o_Tamara%20Paix%c3%a3o%20de%20Oliveira_2013.pdf
Loading triplas: gasto_energetico_em_repouso_de_um_grupo_de_pacientes_com_hepatite_cronica_pelo_virus_c_sem_tratamento
Total time: 0 days 00:38:23.271664
Partial time: 0 days 00:00:12.591045
54 - Downloading and extracting: ecos_de_liberdade_a_santidade_de_jaguaripe_entre_os_alcances_e_limites_da_colonizacao_crista from https://repositorio.ufba.br/bitstream/ri/23332/1/Dissertac%cc%a7a%cc%83o_Jamille.pdf
Loading triplas: ecos_de_liberdade_a_santidade_de_jaguaripe_entre_os_alcances_e_limites_da_colonizacao_crista
Total time: 0 days 00:39:06.206139
Partial time: 0 days 00:00:42.934475
5

Multiple definitions in dictionary at byte 0x2313ea for key /Info
Multiple definitions in dictionary at byte 0x2313f7 for key /Info
Multiple definitions in dictionary at byte 0x231404 for key /Info


Loading triplas: tendencias_e_contradicoes_na_formacao_social_e_economica_do_extremo_sul_da_bahia_entre_1950_e_2000
Total time: 0 days 01:42:20.522222
Partial time: 0 days 00:00:04.395343
157 - Downloading and extracting: tendencias_e_contradicoes_na_formacao_social_e_economica_do_extremo_sul_da_bahia_entre_1950_e_2000 from https://repositorio.ufba.br/bitstream/ri/39217/1/Disserta%c3%a7%c3%a3o%20Economia%20Gustavo%202001.pdf
Loading triplas: tendencias_e_contradicoes_na_formacao_social_e_economica_do_extremo_sul_da_bahia_entre_1950_e_2000
Total time: 0 days 01:43:42.988801
Partial time: 0 days 00:01:22.466579
No PDF link found for uma_abordagem_morfologica_do_bailado_quadros_amazonicos_do_compositor_francisco_mignone
159 - Downloading and extracting: ser_pataxo_educacao_e_identidade_cultural from https://repositorio.ufba.br/bitstream/ri/11816/1/Dissertacao%20Augusto%20Oliveira.pdf
Loading triplas: ser_pataxo_educacao_e_identidade_cultural
Total time: 0 days 01:44:13.872318
Partial time

unknown widths : 
[0, IndirectObject(5927, 0, 1611348101200)]
unknown widths : 
[0, IndirectObject(5922, 0, 1611348101200)]
unknown widths : 
[0, IndirectObject(5917, 0, 1611348101200)]
unknown widths : 
[0, IndirectObject(5927, 0, 1611348101200)]
unknown widths : 
[0, IndirectObject(5922, 0, 1611348101200)]
unknown widths : 
[0, IndirectObject(5917, 0, 1611348101200)]
unknown widths : 
[0, IndirectObject(5927, 0, 1611348101200)]
unknown widths : 
[0, IndirectObject(5922, 0, 1611348101200)]
unknown widths : 
[0, IndirectObject(5917, 0, 1611348101200)]


Loading triplas: identidades_antirracistas_ecos_e_ressonancias_de_discursos_e_argumentos_antiescravagistas
Total time: 0 days 04:52:52.665191
Partial time: 0 days 00:03:51.677984
325 - Downloading and extracting: etnicidade_e_memoria_entre_quilombolas_em_irara_bahia from https://repositorio.ufba.br/bitstream/ri/8663/1/dissertacao_jucelia.pdf
Loading triplas: etnicidade_e_memoria_entre_quilombolas_em_irara_bahia
Total time: 0 days 04:54:54.439457
Partial time: 0 days 00:02:01.773890
326 - Downloading and extracting: trafico_escravidao_e_liberdade_na_bahia_nos_anos_de_ouro_do_comercio_negreiro_c1680c1790 from https://repositorio.ufba.br/bitstream/ri/31918/1/TESE%20Souza%2c%20Daniele%20Santos%20de.pdf
Loading triplas: trafico_escravidao_e_liberdade_na_bahia_nos_anos_de_ouro_do_comercio_negreiro_c1680c1790
Total time: 0 days 04:58:40.052214
Partial time: 0 days 00:03:45.612759
No PDF link found for escravos_e_libertos_nas_minas_do_rio_de_contas__bahia_seculo_xviii
328 - Downloading and ext

KeyboardInterrupt: 